# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries

import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
import re
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.pipeline import Pipeline, FeatureUnion
from nltk.stem import PorterStemmer


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [14]:
# load data from database

engine = create_engine('sqlite:///ETL_Pipeline.db')
df = pd.read_sql_table('message', engine)
X = df.message
y = df.iloc[:,4:]

In [15]:
y.shape

(26215, 36)

In [3]:
print(X[23])
print(y.iloc[23])

Those people who live at Sibert need food they are hungry.
related                   1
request                   1
offer                     0
aid_related               1
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      1
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                      0
earthquake                0
cold             

### 2. Write a tokenization function to process your text data

In [16]:
def tokenize(text):

    tokens = word_tokenize(text)
#     lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words("english"))
    ps = PorterStemmer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = ps.stem(tok).lower().strip()
        if clean_tok.isalpha() and clean_tok not in stop_words:
            clean_tokens.append(clean_tok)

    return clean_tokens

In [23]:
# Test the function 

for i in range(5):
    print(X[i])
    print(tokenize(X[i]))
    print("----------------------------------------")
    
    

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti']
----------------------------------------
Is the Hurricane over or is it not over
['hurrican']
----------------------------------------
Looking for someone but no name
['look', 'someon', 'name']
----------------------------------------
UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogan', 'destroy', 'onli', 'hospit', 'croix', 'function', 'need', 'suppli', 'desper']
----------------------------------------
says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'countri', 'today', 'tonight']
----------------------------------------


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier


pipeline = Pipeline([
    
    ("vect" , CountVectorizer(tokenizer=tokenize)),
    ("tfidf" , TfidfTransformer()),
    ("clf" , MultiOutputClassifier(RandomForestClassifier()))

])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=.15)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [26]:
y_pred = pipeline.predict(X_test)

In [29]:
from sklearn.metrics import classification_report


for i in range(36):
    print(" for {}:".format(y_test.columns[i]))
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))
    
    

 for related:
             precision    recall  f1-score   support

          0       0.64      0.47      0.55       931
          1       0.85      0.92      0.88      3002

avg / total       0.80      0.81      0.80      3933

 for request:
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      3265
          1       0.79      0.40      0.53       668

avg / total       0.87      0.88      0.86      3933

 for offer:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      3915
          1       0.00      0.00      0.00        18

avg / total       0.99      1.00      0.99      3933

 for aid_related:
             precision    recall  f1-score   support

          0       0.74      0.86      0.80      2312
          1       0.74      0.58      0.65      1621

avg / total       0.74      0.74      0.74      3933

 for medical_help:
             precision    recall  f1-score   support

         

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [30]:
# from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# It tooks ages to execute the random cv..

parameters = {
    'clf__estimator__n_estimators': [50, 100,200],
    'clf__estimator__min_samples_split': [2, 3,4]
}

cv = RandomizedSearchCV(pipeline, param_distributions=parameters, n_iter=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.fit(X_train, y_train)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [12]:
from sklearn.ensemble import AdaBoostClassifier

pipeline_2 = Pipeline([
    ("vect", CountVectorizer(tokenizer=tokenize)),
    ("tfidf", TfidfTransformer()),
    ("clf", MultiOutputClassifier(AdaBoostClassifier()))
])


In [ ]:
pipeline_2.fit(X_train, y_train)
y_pred = pipeline_2.predict(X_test)

In [ ]:
for i in range(34):
    print(" for {}:".format(y_test.columns[i]))
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

### 9. Export your model as a pickle file

In [ ]:
import pickle
filename = 'model.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.